In [ ]:
import zipfile

f = zipfile.ZipFile("./Image_NII.zip",'r') # 原压缩文件在服务器的位置
for file in f.namelist():
    f.extract(file,"./Image_NII")               # 解压到的位置
f.close()

In [1]:
import torch.nn.parallel
import torch.optim
import torch.utils.data
import torch.utils.data.distributed
from torch.autograd import Variable
from PIL import Image
from model_34 import generate_model
import torch
import numpy as np
from torch import nn
from torch import optim
from torch.utils.data import DataLoader
#from utils.logger import log
import os
#from test_CT import test
import shutil
import torch.nn.functional as F
from setting_CT_test import parse_opts
from CT_preprocessing_no import CTDataset
import random
from sklearn.metrics import precision_score,f1_score,recall_score

In [2]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
DEVICE

device(type='cuda')

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
def setup_seed(seed):
    Image.MAX_IMAGE_PIXELS = 1000000000000000
    torch.manual_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.enabled = True
setup_seed(0)

In [5]:
sets = parse_opts()
# 删除隐藏文件/文件夹
for root, dirs, files in os.walk('./trails'):
    for file in files:
        if 'ipynb_checkpoints' in file:
            os.remove(os.path.join(root, file))
    if 'ipynb_checkpoints' in root:
        shutil.rmtree(root)

In [6]:
sets.pretrain_path=r'C:/Users/abc/Desktop/MedicalNet_pytorch_files/pretrain/resnet_34_23dataset.pth'

In [ ]:
model, parameters = generate_model(sets)
criterion = nn.CrossEntropyLoss() #交叉熵损失函数
#num_ftrs = model.module.fc.in_features
# model.fc = nn.Linear(num_ftrs, 10)
#model.fc = nn.Sequential(
    #nn.Linear(num_ftrs, 4),
    #nn.ReLU(),
    #nn.Dropout(0.4),
    #nn.Linear(224, 4),
    #nn.LogSoftmax(dim=1)
#)

params = [
    {'params': parameters['base_parameters'], 'lr': sets.learning_rate},
    {'params': parameters['new_parameters'], 'lr': sets.learning_rate * 100}
    ]
device=torch.device('cuda')#注意一下这个跑了之后似乎网络有一点变化
model.to(DEVICE)
# 选择简单暴力的Adam优化器，学习率调低
optimizer = torch.optim.Adam(params, betas=(0.9, 0.999), eps=1e-3)
scheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.99)
print(model)

In [7]:
def train(train_data_loader, model, optimizer, scheduler, total_epochs, save_interval, save_folder, sets):
    scheduler.step()
    loss_train,labels_train,output_train = [],[],[]
    for batch_id, batch_data in enumerate(train_data_loader):
        volumes, labels_ji = batch_data
        labeltry=labels_ji.tolist()
        for i in range(len(labeltry)):
            labels_train.append(labeltry[i])
        if not sets.no_cuda:
            volumes = volumes.cuda()
            labels = labels_ji.cuda().long()
        im = Variable(volumes)
        output = model(volumes)
        loss_value_cls = criterion(output, labels)
        loss = loss_value_cls
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
            # print_loss = loss.data.item()
            # sum_loss += print_loss
        real_loss=loss.item()
        realloss=real_loss*len(batch_data)
        loss_train.append(realloss)
        out_t1 = output.argmax(dim=1)  # 取出预测的最大值
            #result.append(out_t1)
        out_t = out_t1.cpu().tolist()
        for i in range(len(out_t)):
            output_train.append(out_t[i])
            #print('Train Epoch: {} [{}/{} ({:.0f}%)]'.format(epoch, (batch_id + 1) * len(volumes), len(train_data_loader.dataset),100. * (batch_id + 1) / len(train_data_loader)))
        # print('epoch:{}, train_acc: {}, loss:{},precision:{}'.format(epoch, f1, recall,precision))
            
    return labels_train,output_train,loss_train

In [8]:
def val(val_data_loader, model, sets):
    # batch_id_sp = epoch * batches_per_epoch
    model.eval()
    labels_val,out_val,val_loss=[],[],[]
    with torch.no_grad():
        for batch_id, batch_data in enumerate(val_data_loader):
            # data, target = Variable(data).to(device), Variable(target).to(device)
            volumes, label_val = batch_data
            labelval=label_val.tolist()
            for i in range(len(labelval)):
                labels_val.append(labelval[i])
            if not sets.no_cuda:
                volumes = volumes.cuda()
                label = label_val.cuda().long()
            output = model(volumes)
            # output.load_state_dict(checkpoint['state_dict'])#
            loss_value_cls = criterion(output, label)
            loss = loss_value_cls
            val_loss.append(loss.item())
            y_result = output.argmax(dim=1)
            y_result = y_result.cpu().tolist()
            for i in range(len(y_result)):
                out_val.append(y_result[i])
            #print('val Epoch: {} [{}/{} ({:.0f}%)]'.format(epoch, (batch_id + 1) * len(volumes), len(val_data_loader.dataset),100. * (batch_id + 1) / len(val_data_loader)))

    return labels_val,out_val,val_loss

In [9]:
def test(test_data_loader, model, sets):
    # batch_id_sp = epoch * batches_per_epoch
    model.eval()
    labels_test,out_test=[],[]
    for batch_id, batch_data in enumerate(test_data_loader):
            # data, target = Variable(data).to(device), Variable(target).to(device)
        volumes, label_test = batch_data
        labeltest=label_test.tolist()
        for i in range(len(labeltest)):
            labels_test.append(labeltest[i])
        if not sets.no_cuda:
            volumes = volumes.cuda()
            label = label_test.cuda().long()
            output = model(volumes)
            # output.load_state_dict(checkpoint['state_dict'])#
            out=output.argmax(dim=1)
            out = out.cpu().tolist()
            for i in range(len(out)):
                out_test.append(out[i])
    return labels_test,out_test

In [10]:
if sets.resume_path:
    if os.path.isfile(sets.resume_path):
        print("=> loading checkpoint '{}'".format(sets.resume_path))
        checkpoint = torch.load(sets.resume_path)
        model.load_state_dict(checkpoint['state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer'])
        print("=> loaded checkpoint '{}' (epoch {})"
          .format(sets.resume_path, checkpoint['epoch']))

# getting data
sets.phase = 'train'
if sets.no_cuda:
    sets.pin_memory = False
else:
    sets.pin_memory = True

In [11]:
sets.batch_size=4

In [12]:
total_epochs=sets.n_epochs
        
for i in range(5):
    for d in range(4):
        sets.img_list_train=[]
        sets.img_list_val=[]
        sets.img_list_test=[]
        training_dataset=[]
        val_dataset=[]
        test_dataset=[]
        f1_val_list=[]
        torch.manual_seed(sets.manual_seed)
        #model, parameters = generate_model(sets)
        model,parameters = generate_model(sets)
        #params = [{ 'params': parameters['base_parameters'], 'lr': sets.learning_rate },{ 'params': parameters['new_parameters'], 'lr': sets.learning_rate*100 }]
        criterion = nn.CrossEntropyLoss() #交叉熵损失函数
        optimizer = torch.optim.Adam(parameters, betas=(0.9, 0.999), eps=1e-2)
        scheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.99)
        device=torch.device('cuda')#注意一下这个跑了之后似乎网络有一点变化
        model.to(DEVICE)
        # 选择简单暴力的Adam优化器，学习率调低
        sets.img_list_train=('./data_list/train_fold{}f{}.txt'.format(i,d))
        sets.img_list_val = ('./data_list/val_fold{}f{}.txt'.format(i,d))
        sets.img_list_test = ('./data_list/test_fold{}f{}.txt'.format(i,d))
        print("it's training in fold_{}_{}".format(i,d))
        training_dataset = CTDataset(sets.data_root, sets.img_list_train, sets, phase='train')
        print('train len is:',len(training_dataset))
        train_data_loader = DataLoader(training_dataset, batch_size=sets.batch_size, shuffle=True, num_workers=sets.num_workers,
                                 pin_memory=sets.pin_memory)
        val_dataset = CTDataset(sets.data_root, sets.img_list_val, sets, phase='train')
        val_data_loader = DataLoader(val_dataset, batch_size=sets.batch_size, shuffle=True, num_workers=sets.num_workers,
                                 pin_memory=sets.pin_memory)
        for epoch in range(1, total_epochs + 1):
            labels,output,loss = [],[],[]
            labels,output,loss = train(train_data_loader,model, optimizer, scheduler,total_epochs, save_interval=sets.save_intervals,
                                          save_folder=sets.save_folder, sets=sets)   
            f1 = f1_score(labels, output, average='macro')
            precision = precision_score(labels, output, average='macro')
            recall = recall_score(labels, output, average='macro')
            loss_train=sum(loss)/len(train_data_loader)
            print('Train Epoch{}:f1: {:.6f}\trecall: {:.6f}\tprecision: {:.6f}\tloss: {:.6f}'.format(epoch,f1,recall,precision,loss_train))
            #labels_val,out_val,val_loss=[],[],[]
            labels_val,out_val,val_loss = val(val_data_loader, model, sets)
            f1_val = f1_score(labels_val,out_val, average='macro')
            f1_val_list.append(f1_val)
            precision_val = precision_score(labels_val,out_val, average='macro')
            recall_val = recall_score(labels_val,out_val, average='macro')
            loss_val=sum(val_loss)/len(val_data_loader)
            print('Val Epoch:f1: {:.6f}\trecall: {:.6f}\tprecision: {:.6f}\tloss: {:.6f}'.format(f1_val,recall_val,precision_val,loss_val))
        
            if f1_val == max(f1_val_list):    #那个列表下标有点问题
                #if batch_id_sp != 0 and batch_id_sp % save_interval == 0:
                    model_save_path = './trails/models/fold_{}.pth.tar'.format(i)
                    model_save_dir = os.path.dirname(model_save_path)
                    if not os.path.exists(model_save_dir):
                        os.makedirs(model_save_dir)

                    torch.save({
                                'fold':i,
                                'state_dict': model.state_dict(),
                                'optimizer': optimizer.state_dict()},
                                model_save_path)

        sets.phase='test'
        checkpoint=torch.load(model_save_path)
        net,_=generate_model(sets)
        net.load_state_dict(checkpoint['state_dict'])

        # data tensor
        test_dataset = CTDataset(sets.data_root, sets.img_list_test, sets, phase='test')
        test_data_loader = DataLoader(test_dataset, batch_size=sets.batch_size, shuffle=True, num_workers=sets.num_workers,pin_memory=sets.pin_memory)
        #label_test,out_test=[],[]
        label_test,out_test=test(test_data_loader,net,sets)
        f1_test = f1_score(label_test, out_test, average='macro')
        recall_test = recall_score(label_test, out_test, average='macro')
        precision_test = precision_score(label_test, out_test, average='macro')
        print('Test Epoch:f1: {:.6f}\trecall: {:.6f}\tprecision: {:.6f}'.format(f1_test,recall_test,precision_test))

loading pretrained model C:/Users/abc/Desktop/MedicalNet_pytorch_files/pretrain/resnet_34_23dataset.pth
it's training in fold_0_0
Processing 81 datas
train len is: 81
Processing 27 datas
Train Epoch1:f1: 0.184481	recall: 0.215260	precision: 0.161409	loss: 3.134249
Val Epoch:f1: 0.000000	recall: 0.000000	precision: 0.000000	loss: 7.283732
Train Epoch2:f1: 0.189044	recall: 0.201698	precision: 0.194481	loss: 12.140556
Val Epoch:f1: 0.205128	recall: 0.333333	precision: 0.148148	loss: 1.690448
Train Epoch3:f1: 0.155556	recall: 0.225649	precision: 0.133990	loss: 3.029834
Val Epoch:f1: 0.152381	recall: 0.333333	precision: 0.098765	loss: 1.200107
Train Epoch4:f1: 0.194602	recall: 0.212488	precision: 0.181250	loss: 2.822312
Val Epoch:f1: 0.205128	recall: 0.333333	precision: 0.148148	loss: 1.298242
Train Epoch5:f1: 0.138889	recall: 0.214286	precision: 0.102740	loss: 2.699800
Val Epoch:f1: 0.205128	recall: 0.333333	precision: 0.148148	loss: 1.265309
Train Epoch6:f1: 0.195767	recall: 0.236688	prec

KeyboardInterrupt: 